In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import nltk

from nlu_engine import NLUEngine
from nlu_engine import MacroDataRefinement
from nlu_engine import MacroEntityRefinement

from nlu_engine import DataUtils
from nlu_engine import RenderJSON

from nlu_engine import Analytics

from nlu_engine import EntityExtractor, crf

# Macro NLU Data Refinement

It's a bit like the TV show [Serverance](https://www.imdb.com/title/tt11280740/) .

![Helly R and Mark S](https://media.npr.org/assets/img/2022/02/15/atv_severance_photo_010103-5f8033cc2b219ba64fe265ce893eae4c90e83896-s1100-c50.jpg "Helly R and Mark G")

*Helly R*: `My job is to scroll through the spreadsheet and look for the numbers that feel scary?`

*Mark S*: `I told you, you’ll understand when you see it, so just be patient.`

![MDR](https://www.imore.com/sites/imore.com/files/styles/large/public/field/image/2022/03/refinement-software-severance-apple-tv.jpg "serverance micro data refinement")

*Helly R*: `That was scary. The numbers were scary.`

Hopefully the intents and entities that are wrong aren't scary, just a bit frustrating. Let's see if we can find the right ones.

NOTE: We will use Logistic Regression with TFIDF features to train our intent models and CRFs for entity exraction. Why? Well, they are very fast and both methods aren't state-of-the-art. This is good, because it is easier to find problems we will need to refine in the dataset than if we were to use a proper NLU engine like Snips or something SOTA like BERT. It is very important to note that some of the the problems we will pick up on, might not be an actual issue, but might be due to the limitations of the models. Refining the real problems and ignoring the limitations of the models is a good way to improve the models. Then when the dataset is ready, we can use some more advanced NLU engine and get the best performance possible.

* Macro NLU Data Refinement: Intent
* Macro NLU Data Refinement: Entity


Load the dataset

In [2]:
try:
    nlu_data_df = pd.read_csv(
        'data/refined/nlu_data_refined_df.csv', sep=',', index_col=0)
    print('Successfully loaded nlu_data_refined_df.csv')
except:
    data = 'data/NLU-Data-Home-Domain-Annotated-All-Cleaned.csv'
    nlu_data_df = DataUtils.load_data(
    data
)

Successfully loaded nlu_data_refined_df.csv


In [ ]:
# TODO: Remove this when done.It's just for testing!
data = 'data/NLU-Data-Home-Domain-Annotated-All-Cleaned.csv'
nlu_data_df = DataUtils.load_data(
    data
)

Make sure `nlu_data_df['answer_normalised']` is taken from `nlu_data_df['answer_annotation']`

In [3]:
nlu_data_df = DataUtils.convert_annotated_utterances_to_normalised_utterances(
    nlu_data_df)


We should remove the unwanted entries for the next few steps.

In [4]:
removed_nlu_data_refined_df = nlu_data_df[nlu_data_df['remove'] != True]

# Entity extraction report

The entity extraction could be greatly improved by improving the features it uses. It would be great if someone would take a look at this. Perhaps the CRF features similar to what Snips uses would be better such as Brown clustering (probably).

In [ ]:
#TODO: implement brown clustering to improve entity extraction (see entity_extractor.py)

It is important to have the NLTK tokenizer to be able to extract entities.

In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
        nltk.download('punkt')

Due to this error featured in [this git issue](https://github.com/TeamHG-Memex/sklearn-crfsuite/issues/60) we have to use an older version of scikit learn (sklearn<0.24), otherwise the latest version would work. Hopefully this gets fixed one day..

In [ ]:
entity_report_df = NLUEngine.evaluate_entity_classifier(
    data_df=removed_nlu_data_refined_df, cv=4)

In [ ]:
entity_report_df

In [8]:
removed_nlu_data_refined_df['entity_types'] = removed_nlu_data_refined_df['answer_annotation'].apply(
    EntityExtractor.extract_entities)

/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
removed_nlu_data_refined_df

,userid,answerid,notes,question,suggested_entities,answer,answer_normalised,scenario,intent,predicted_label,intent_refined,entity_refined,remove,status,answer_annotation,entity_types
0,1.0,1.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 5am this week,wake me up at five am this week,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : five am] [date : this week],"[{'type': 'time', 'words': ['five', 'am']}, {'..."
1,1.0,2.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 9am on Friday,wake me up at nine am on friday,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : nine am] on [date : friday],"[{'type': 'time', 'words': ['nine', 'am']}, {'..."
2,1.0,3.0,NaN,Write what you would tell your PDA in the foll...,"date, time",set an alarm for two hours from now,set an alarm for two hours from now,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,set an alarm for [time : two hours from now],"[{'type': 'time', 'words': ['two', 'hours', 'f..."
3,1.0,31.0,NaN,Write what you would tell your PDA in the foll...,player_setting,Olly quiet!,quiet,audio,volume_mute,volume_mute,NaN,NaN,NaN,NaN,quiet,[]
7,1.0,38.0,NaN,How would you ask your PDA to change the light...,color_type,make the lighting bit more warm here,make the lighting a bit more warm here,iot,hue_lightchange,hue_lightup,True,NaN,False,NaN,make the lighting a bit more [color_type : war...,"[{'type': 'color_type', 'words': ['warm']}]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25711,NaN,781.0,NaN,NaN,NaN,"that's cool, musch appreciated, olly.","that's cool, musch appreciated, olly.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, musch appreciated, olly.",[]
25712,NaN,782.0,NaN,NaN,NaN,"you are hero, appreciated.","you are hero, appreciated.",general,praise,praise,NaN,NaN,NaN,NaN,"you are hero, appreciated.",[]
25713,NaN,783.0,NaN,NaN,NaN,"thanks, that's nice.","thanks, that's nice.",general,praise,praise,NaN,NaN,NaN,NaN,"thanks, that's nice.",[]
25714,NaN,784.0,NaN,NaN,NaN,"that's cool, thank you so much.","that's cool, thank you so much.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, thank you so much.",[]


In [7]:
# TODO: instead of getting domains for each entity type,
# get entries for each domain and create classification report for each domain
domain = 'general'
domain_df = removed_nlu_data_refined_df[removed_nlu_data_refined_df['scenario'] == domain]
#domain_entity_report_df = NLUEngine.evaluate_entity_classifier(data_df=domain_df, cv=4)

In [ ]:
domain_df['entity_types'] = domain_df['answer_annotation'].apply(EntityExtractor.extract_entities)


In [ ]:
import regex as re

In [ ]:
domain_df['entity_types'] = domain_df['answer_annotation'].apply(
    lambda utterance: re.findall(r'\[(.*?)\]', utterance))
#TODO: have to find a way to get the entity type from the utterance


In [ ]:
domain_df.head(50)

In [ ]:
#TODO: get all domains, get overall scores and combine them into a single report
domain_entity_report_df
#TODO: graph this report
#TODO: user picks a domain, then they see this report of the entity breakdown.
# TODO: the user gets a data sheet for the domain arranged by support.

In [ ]:
# Refactor method to include domain as a possible input
Analytics.plot_report(domain_entity_report_df)

In [ ]:
domains = removed_nlu_data_refined_df['scenario'].unique().tolist()


In [ ]:
domains

In [ ]:
def get_entity_reports_for_domains(data_df):
    domains = data_df['scenario'].unique().tolist()
    domain_entity_reports_df = pd.DataFrame()
    for domain in domains:
        print(f'Evaluating entity classifier for {domain}')
        domain_df = data_df[data_df['scenario'] == domain]
        try:
            domain_entity_report_df = NLUEngine.evaluate_entity_classifier(
                data_df=domain_df, cv=3)
            domain_scores_df = domain_entity_report_df.tail(3)
            domain_scores_df['domain'] = domain

            domain_entity_reports_df.append(
                domain_scores_df)

        except Exception as e:
            print(f'Error evaluating entity classifier for {domain}')
            print(e)

    return domain_entity_reports_df

domain_entity_reports_df = get_entity_reports_for_domains(domain_df)


In [ ]:
X, y = EntityExtractor.get_targets_and_labels(domain_df)

In [ ]:
model = EntityExtractor.train_crf_model(X, y)

In [ ]:
y_pred = EntityExtractor.predict_crf_model(model, X)

In [ ]:
y_pred

In [ ]:
predictions = Analytics.cross_validate_classifier(crf, X, y, cv=2)

In [ ]:
NLUEngine.evaluate_entity_classifier(
    data_df=domain_df, cv=3)

In [ ]:
domain_entity_reports_df = NLUEngine.get_entity_reports_for_domains(removed_nlu_data_refined_df)

In [ ]:
domain_entity_reports_df

In [ ]:
#TODO: Extract the entities into something countable!

In [ ]:
crf_model = NLUEngine.train_entity_classifier(removed_nlu_data_refined_df)

In [ ]:
model_path = 'models/analytics/entity_tagger.sav'

In [ ]:
DataUtils.pickle_model(classifier=crf_model, model_path=model_path)

In [ ]:
crf_model = DataUtils.import_pickled_model(model_path)

In [ ]:
annotated_utterance = 'set an alarm for [time : two hours] [time : from now]'


In [ ]:
def combine_neigboring_same_entity_types(utterance, crf_model):
    """
    Combines neighboring entities of the same type and removes them in an utterance, until all of the same neigboring entities are combined
    """
    tagged_utterance = EntityExtractor.tag_utterance(
        utterance,
        crf_model
    )
    split_tagged_utterance = tagged_utterance.split(' ')

    def combine_and_remove_entities(split_tagged_utterance):
        """
        Combines neighboring entities of the same type and marks the duplicates to be removed.
        NOTE: It is important for parsing to keep the same length, therefore we mark them instead of directly remove the matches.
        """
        change_counter = 0
        for index, token in enumerate(split_tagged_utterance):
            if '[' in token:
                if len(split_tagged_utterance) > index + 3:
                    if token == split_tagged_utterance[index + 3]:
                        split_tagged_utterance[index + 2] = split_tagged_utterance[index + 2].replace(
                            ']', '') + ' ' + split_tagged_utterance[index + 5]

                        split_tagged_utterance[index + 3] = 'to_remove'
                        split_tagged_utterance[index + 4] = 'to_remove'
                        split_tagged_utterance[index + 5] = 'to_remove'
                        change_counter += 1
        return (split_tagged_utterance, change_counter)

    def remove_entities(split_tagged_utterance):
        try:
            while True:
                split_tagged_utterance.remove('to_remove')
        except ValueError:
            pass
        return split_tagged_utterance

    combined_entities_split_tagged_utterance, change_counter = combine_and_remove_entities(
        split_tagged_utterance)
    removed_entities_split_tagged_utterance = remove_entities(
        combined_entities_split_tagged_utterance)

    while change_counter > 0:
        combined_entities_split_tagged_utterance, change_counter = combine_and_remove_entities(
            removed_entities_split_tagged_utterance)
        removed_entities_split_tagged_utterance = remove_entities(
            combined_entities_split_tagged_utterance)

    return ' '.join(removed_entities_split_tagged_utterance)


In [ ]:
combine_neigboring_same_entity_types(utterance, crf_model)

In [ ]:
removed_nlu_data_refined_df

In [ ]:
removed_nlu_data_refined_df['predicted_tagging'] = removed_nlu_data_refined_df['answer_normalised'].apply(
    lambda x: combine_neigboring_same_entity_types(x, crf_model))

In [ ]:
removed_nlu_data_refined_df[removed_nlu_data_refined_df['answer_annotation']
                            != removed_nlu_data_refined_df['predicted_tagging']]
# TODO: have user choose a domain, make report about entities in the domain,...
# TODO: after the report, does the user clean all entities in the domain at once, or by individual entity type?


In [ ]:
removed_nlu_data_refined_df.loc[21, 'answer_annotation']


In [ ]:
#TODO: It is probably a good idea to drop all of the ones that lack a good support.
#NOTE: But it didn't work to fix the problem.
remove_entities = [
    'music_album',
    'game_type',
    
]
removed_nlu_data_refined__entities_cleaned_df = removed_nlu_data_refined_df[~removed_nlu_data_refined_df['answer_annotation'].str.contains('|'.join(remove_entities))]

In [ ]:
Analytics.plot_report(entity_report_df)

In [ ]:
#TODO: Remove/replace worst: add in state features like here: https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#let-s-check-what-classifier-learned
# Specifically, we want print_state_features()

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))


In [ ]:
from nlu_engine import crf
from collections import Counter

In [ ]:
crf_model.state_features_

In [ ]:
print_state_features(Counter(crf.state_features_).most_common(100))

In [ ]:
# TODO: review the most common features, none of them are the word parts (chunks) or POS tags, are these even needed or helpful?
# Can we remove them and speed up the process?
# What other features could be used? Word2vec? Brown clustering?

In [ ]:
domain_selection = MacroDataRefinement.list_and_select_domain(nlu_data_df)

As we have seen from the entity extraction report, the entity extraction is not working for the alarm_type.

In [ ]:
#TODO: review all scoring 0, see if they can be completely dropped or what
entity_to_refine = 'alarm_type'
nlu_scenario_df = removed_nlu_data_refined_df[removed_nlu_data_refined_df['answer_annotation'].str.contains(
    entity_to_refine)]

In [ ]:
nlu_scenario_df

In [ ]:
def remove_entity(df, entity_to_remove):
    """
        Remove all entries of an entity type from the dataframe.
        :param df: pandas dataframe
        :return: pandas dataframe
        """
    updated_df = df.copy()
    updated_df.loc[updated_df['answer_annotation'].str.contains(
        entity_to_remove), 'remove'] = True
    return updated_df


In [ ]:
updated_df = remove_entity(removed_nlu_data_refined_df, entity_to_refine)

In [ ]:
removed_nlu_data_refined_df[removed_nlu_data_refined_df['answer_annotation'].str.contains(
    entity_to_refine)]

## Entity Convert to ipysheet and review
TODO: add in description of the types of fixes we can do to the NLU data for entity


In [ ]:

# TODO: same as above for intents but with predicted entities: report on them, break them down into a dictionary of dataframes and refine them..

For the example with 'alarm' and the alarm_type:
* We see that the alarm_type entities are really event_name(ie wake up, soccer practice) except for ID 5879, we will need to change them to event_name and remove ID 5879.
* The last one(ID 6320) is a mistake. Someone got confused with the prompt and assumed alarm is a security system. This is out of scope for the alarm domain, as the alarms are ones set on a phone or other device. We will drop this utterance.
Once you are done reviewing, you convert it back to a dataframe and check to make sure it looks okay.
Let's change all alarm_type entities to event_name.

In [ ]:

reviewed_scenario_df['answer_annotation'] = reviewed_scenario_df['answer_annotation'].str.replace(
    'alarm_type', 'event_name')
reviewed_scenario_df


Okay dokey, now we can merge this with the original data set and see if it made a difference already(well of course it did!).

In [ ]:
nlu_data_df.drop(
    reviewed_scenario_df[reviewed_scenario_df['remove'] == True].index, inplace=True)

reviewed_scenario_df = reviewed_scenario_df[~reviewed_scenario_df['remove'] == True]

nlu_data_df.loc[nlu_data_df.index.intersection(
    reviewed_scenario_df.index), 'answer_annotation'] = reviewed_scenario_df['answer_annotation']

nlu_data_df[(nlu_data_df['scenario'].str.contains('alarm')) & (nlu_data_df['answer_annotation'].str.contains(
    'event_name'))]


### Benchmark changed data set
TODO: repeat reports for the changed data set for domain and entities and compare


In [ ]:

entity_reviewed_report_df = NLUEngine.evaluate_entity_classifier(
    data_df=nlu_data_df)
entity_reviewed_report_df.sort_values(by=['f1-score'])

If you are sure it is okay, you can save it as a csv file, make sure to name it correctly(i.e. `alarm_domain_first_review.csv`)

In [ ]:
reviewed_scenario_df.to_csv('alarm_domain_first_review.csv')

Load it back up and check to make sure it looks okay. Make sure to give it the right name!


In [ ]:
audio_domain_first_review_df = pd.read_csv(
    'alarm_domain_first_review.csv', index_col=0)
audio_domain_first_review_df.tail(50)


In [ ]:
# TODO: implement the evaluate_classifier in the NLU engine to check f1 score for intents and entities in the domain vs original NLU data of domain!
# Value: benchmark!
#TODO: implement a flow for getting the domains with the lowest f1 scores by intent/domain and entities and cleaning them by the order of the lowest f1 scores
# TODO: concat all reviewed dfs and save to csv
# TODO: add benchmark for whole NLU data set before and after cleaning! (by intents and domains!)
# TODO: review the review marked entries
# TODO: add new column for notes
# TODO: change flow of review for only ones that should be reviewed, not all of the ones that have been changed (track changes by comparing against the original data set)
# TODO: do the changed utterances have to be changed in other fields too or is it just enough for the tagged utterancve field?
# TODO: add visualizations of domains, their intents, keywords in utterances, and entities to top
